In [1]:
import trimet_plan_game_helper_scripts as helpers
import geopandas as gpd
import boto3
from shapely.geometry import LineString
import pandas as pd

In [2]:
tm_boundary = gpd.read_file("gis/export/tm_route_buffer_bounds.geojson")
trimet_crs = "EPSG:2913"

In [3]:
gdf_points, itineraries_reduced, itinerary_routes_reduced, tries = helpers.generate_random_points_make_itinerary(tm_boundary, trimet_crs)

In [4]:
itineraries_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
0,0,8552,2569,5400,583,3196.218746,0,WALK,WALK,,Origin,2346,SW Halsey & Halsey Loop,"LINESTRING (-122.40032 45.53401, -122.39993 45...",77 to 12 to 44
1,0,8552,2569,5400,583,3196.218746,1,77,BUS,2346,SW Halsey & Halsey Loop,7484,NE Cesar Chavez Blvd & Sandy,"LINESTRING (-122.39670 45.53913, -122.39689 45...",77 to 12 to 44
2,0,8552,2569,5400,583,3196.218746,2,WALK,WALK,7484,NE Cesar Chavez Blvd & Sandy,11313,NE Sandy & Cesar Chavez Blvd,"LINESTRING (-122.62289 45.53469, -122.62289 45...",77 to 12 to 44
3,0,8552,2569,5400,583,3196.218746,3,12,BUS,11313,NE Sandy & Cesar Chavez Blvd,173,SW Barbur & Hamilton,"LINESTRING (-122.62269 45.53495, -122.62280 45...",77 to 12 to 44
4,0,8552,2569,5400,583,3196.218746,4,44,BUS,173,SW Barbur & Hamilton,1957,SW Garden Home & 45th,"LINESTRING (-122.67850 45.49096, -122.67854 45...",77 to 12 to 44
5,0,8552,2569,5400,583,3196.218746,5,WALK,WALK,1957,SW Garden Home & 45th,,Destination,"LINESTRING (-122.72280 45.46563, -122.72283 45...",77 to 12 to 44
6,1,8552,2697,4880,975,3364.642746,0,WALK,WALK,,Origin,2346,SW Halsey & Halsey Loop,"LINESTRING (-122.40032 45.53401, -122.39993 45...",77 to 200 to 44
7,1,8552,2697,4880,975,3364.642746,1,77,BUS,2346,SW Halsey & Halsey Loop,10846,NE Jonesmore & 82nd,"LINESTRING (-122.39670 45.53913, -122.39689 45...",77 to 200 to 44
8,1,8552,2697,4880,975,3364.642746,2,WALK,WALK,10846,NE Jonesmore & 82nd,8371,NE 82nd Ave MAX Station,"LINESTRING (-122.57885 45.53402, -122.57884 45...",77 to 200 to 44
9,1,8552,2697,4880,975,3364.642746,3,200,TRAM,8371,NE 82nd Ave MAX Station,7618,PSU Urban Center/SW 5th & Mill MAX Station,"LINESTRING (-122.57843 45.53349, -122.57852 45...",77 to 200 to 44


In [5]:
gdf_points.to_file("gis/export/origin_destination_points.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_points.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_points.geojson",ExtraArgs={'ACL':'public-read'})

In [6]:
itineraries_reduced.to_file("gis/export/itineraries_reduced.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itineraries_reduced.geojson", "meysohn-sandbox", "trimet_trip_planner/itineraries_reduced.geojson",ExtraArgs={'ACL':'public-read'})

In [7]:
itinerary_routes_reduced.to_file("gis/export/itinerary_routes_reduced.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itinerary_routes_reduced.geojson", "meysohn-sandbox", "trimet_trip_planner/itinerary_routes_reduced.geojson",ExtraArgs={'ACL':'public-read'})

In [8]:
origin_destination_centriod = LineString(list(gdf_points['points'].to_numpy())).centroid

In [9]:
origin_destination_centriod_gdf = gpd.GeoDataFrame(pd.DataFrame(['origin_dest_centroid'], columns=['id']),crs="EPSG:4326", geometry=[origin_destination_centriod])
origin_destination_centriod_gdf

,id,geometry
0,origin_dest_centroid,POINT (-122.57056 45.50360)


In [10]:
origin_destination_centriod_gdf.to_file("gis/export/origin_destination_centriod.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_centriod.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_centriod.geojson",ExtraArgs={'ACL':'public-read'})